In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Monthly_Ridership___Traffic_Data__Beginning_January_2008.csv
/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Daily_Ridership_Data__Beginning_2020.csv
/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload


In [2]:
monthly_riders_since_2008 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Monthly_Ridership___Traffic_Data__Beginning_January_2008.csv')

#No missing values
monthly_riders_since_2008.dropna(inplace=True)

#Filter to only include Subway and no other MTA agency
monthly_riders_since_2008 = monthly_riders_since_2008[monthly_riders_since_2008['Agency'] == 'Subway'] 

#Rename "Month" column into Date
monthly_riders_since_2008.rename(columns = {'Month':'Date'}, inplace = True)

#Polish Date column
monthly_riders_since_2008['Date'] = monthly_riders_since_2008['Date'].str.replace('/01/', '/', regex=True)

#Splitting Month column into Year and Month values
monthly_riders_since_2008[['Month', 'Year']] = monthly_riders_since_2008['Date'].str.split('/', expand=True)



monthly_riders_since_2008

,Date,Agency,Ridership,Month,Year
238,01/2018,Subway,134683435,01,2018
242,02/2018,Subway,127432835,02,2018
246,03/2018,Subway,143982923,03,2018
251,04/2018,Subway,141950369,04,2018
255,05/2018,Subway,150320833,05,2018
...,...,...,...,...,...
686,10/2023,Subway,103949018,10,2023
693,11/2023,Subway,98511566,11,2023
700,12/2023,Subway,95685578,12,2023
706,01/2024,Subway,91419551,01,2024


In [3]:
daily_riders_since_2020 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Daily_Ridership_Data__Beginning_2020.csv')

#No missing values
daily_riders_since_2020.dropna(inplace=True)

#Drop unccesary columns that belong to other agencies outside of the subway 
daily_riders_since_2020.drop(daily_riders_since_2020.iloc[:, 3:], inplace=True, axis=1)

#Convert to Datetime
daily_riders_since_2020['Date'] = pd.to_datetime(daily_riders_since_2020['Date'])

#Sort by date
daily_riders_since_2020.sort_values('Date', ascending=True, inplace=True)

daily_riders_since_2020

,Date,Subways: Total Estimated Ridership,Subways: % of Comparable Pre-Pandemic Day
1542,2020-03-02,5329915,0.96
1541,2020-03-03,5481103,0.98
1540,2020-03-04,5498809,0.99
1539,2020-03-05,5496453,0.99
1538,2020-03-06,5189447,0.93
...,...,...,...
4,2024-05-19,2121168,0.89
3,2024-05-20,3677281,0.64
2,2024-05-21,4024394,0.70
1,2024-05-22,4096783,0.72


In [4]:
#df3 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload', iterator=True, chunksize=1000)
hourly_ridership_since_2022 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload', nrows=10000)


#dropping missing values
hourly_ridership_since_2022.dropna(inplace=True)

#Convert to Datetime
hourly_ridership_since_2022['transit_timestamp'] = pd.to_datetime(hourly_ridership_since_2022['transit_timestamp'])

#only showing
hourly_ridership_since_2022 = hourly_ridership_since_2022[hourly_ridership_since_2022['borough'] != 'Staten Island'] 

#Drop unccesary columns for our analysis
hourly_ridership_since_2022.drop(columns=['payment_method', 'fare_class_category', 'transfers'], inplace=True, axis=1)


hourly_ridership_since_2022.head()

/tmp/ipykernel_18/1911075595.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hourly_ridership_since_2022['transit_timestamp'] = pd.to_datetime(hourly_ridership_since_2022['transit_timestamp'])


,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,ridership,latitude,longitude,Georeference
0,2023-02-12 18:00:00,subway,164,"34 St-Penn Station (A,C,E)",Manhattan,84,40.752289,-73.993393,POINT (-73.99339294433594 40.752288818359375)
1,2023-03-28 14:00:00,subway,164,"34 St-Penn Station (A,C,E)",Manhattan,123,40.752289,-73.993393,POINT (-73.99339294433594 40.752288818359375)
2,2023-01-11 02:00:00,subway,164,"34 St-Penn Station (A,C,E)",Manhattan,3,40.752289,-73.993393,POINT (-73.99339294433594 40.752288818359375)
3,2023-06-08 17:00:00,subway,164,"34 St-Penn Station (A,C,E)",Manhattan,200,40.752289,-73.993393,POINT (-73.99339294433594 40.752288818359375)
4,2023-07-25 16:00:00,subway,164,"34 St-Penn Station (A,C,E)",Manhattan,872,40.752289,-73.993393,POINT (-73.99339294433594 40.752288818359375)
